## Final Project Submission

Please fill out:
* Student name: Magali Solimano
* Student pace: self paced 
* Scheduled project review date/time: 
* Instructor name: Jeff Herman
* Blog post URL:


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 

In [ ]:
## Load the data (11 files)
df1 = pd.read_csv('zippedData/bom.movie_gross.csv.gz', compression='gzip')
df2 = pd.read_csv('zippedData/imdb.name.basics.csv.gz', compression='gzip')
df3 = pd.read_csv('zippedData/imdb.title.akas.csv.gz', compression='gzip')
df4 = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip') 
df5 = pd.read_csv('zippedData/imdb.title.crew.csv.gz', compression='gzip')
df6 = pd.read_csv('zippedData/imdb.title.principals.csv.gz', compression='gzip')
df7 = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip')
df8 = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t', compression='gzip')
df9 = pd.read_csv('zippedData/rt.reviews.tsv.gz', compression='gzip', delimiter='\t', encoding='iso-8859-1')
df10 = pd.read_csv('zippedData/tmdb.movies.csv.gz', compression='gzip')
df11 = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', compression='gzip')

In [ ]:
# Explore the dataframes and data types
print(df1.info())
df1.head()

In [ ]:
print(df2.info())
df2.head()

In [ ]:
print(df3.info())
df3.head()

In [ ]:
print(df4.info())
df4.head()

In [ ]:
print(df5.info())
df5.head()

In [ ]:
print(df6.info())
df6.head()

In [ ]:
print(df7.info())
df7.head()

In [ ]:
print(df8.info())
df8.head()

In [ ]:
print(df9.info())
df9.head()

In [ ]:
print(df10.info())
df10.head()

In [ ]:
print(df11.info())
df11.head()

In [ ]:
## QUESTION 1: What movies are the top revenue, ROI generators?

In [ ]:
#df1 and #df11 have revenue cols -- compare these results. Can the dfs be joined?

In [ ]:
# Convert budget and revenue cols to float dtype; release_date to datetime. Remove '$' and ','.
df11['production_budget'] = df11['production_budget'].map(lambda x: float(x.replace("$", "").replace(',','')))
df11['domestic_gross'] = df11['domestic_gross'].map(lambda x: float(x.replace("$", "").replace(',','')))
df11['worldwide_gross'] = df11['worldwide_gross'].map(lambda x: float(x.replace("$", "").replace(',','')))
df11['release_date'] = pd.to_datetime(df11['release_date'])
df11['year'] = pd.DatetimeIndex(df11['release_date']).year

# Confirm that dtype conversions worked and that null values did not increase
display(df11.info())

In [ ]:
df11 = df11.rename(columns={"movie": "title"})
df11.head()

In [ ]:
# Compare top gross: df1 vs df11

In [ ]:
display(df1.sort_values(by=['domestic_gross'], ascending=False).head(20))
df11.sort_values(by=['domestic_gross'], ascending=False).head(20)

In [ ]:
# There is overlap between df1 & df11. But, df11 has more movie entries (incl. several top revenue generators) that are not in df1.
# df1 might not have a particular studio's movies.

In [ ]:
# Set index to title.
# Join df11 and df1--left join to bring in df1's studio info. 

In [ ]:
df1.set_index('title', inplace = True)
df1.head()

In [ ]:
df11.set_index('title', inplace = True)
df11.head()

In [ ]:
df_financials = df11.join(df1, how='left', rsuffix='_right')
display(df_financials.info())
df_financials.head()

In [ ]:
# Drop df1 revenue data since they seem to round, while df11's revenue data is more precise.
# Drop foreign_gross, year_right.

In [ ]:
df_financials = df_financials.drop(columns=['domestic_gross_right', 'foreign_gross', 'year_right'])

In [ ]:
# Convert "0" values to NaN
df_financials['production_budget'] = df_financials['production_budget'].replace(0.0, np.nan)
df_financials['domestic_gross'] = df_financials['domestic_gross'].replace(0.0, np.nan)
df_financials['worldwide_gross'] = df_financials['worldwide_gross'].replace(0.0, np.nan)


# Convert values to millions
df_financials['production_budget_m'] = (df_financials['production_budget'] / 1000000).round(1)
df_financials['domestic_gross_m'] = (df_financials['domestic_gross'] / 1000000).round(1)
df_financials['worldwide_gross_m'] = (df_financials['worldwide_gross'] / 1000000).round(1)

df_financials.head(10)

In [ ]:
# Create new cols for foreign_gross, net_revenue, ROI
df_financials['foreign_gross_m'] = (df_financials['worldwide_gross_m'] - df_financials['domestic_gross_m']).round(1)
df_financials['net_revenue_m'] = (df_financials['worldwide_gross_m'] - df_financials['production_budget_m']).round(1)
df_financials['ROI'] = ((df_financials['worldwide_gross'] - df_financials['production_budget']) / df_financials['production_budget']).round(2)
df_financials['ROIpct'] = (((df_financials['worldwide_gross'] - df_financials['production_budget']) / df_financials['production_budget'])*100).round(2)
df_financials.head()

In [ ]:
# reset index
df_financials.reset_index(inplace = True)

In [ ]:
# Check for duplicates. Explore if need to remove or can keep.
df_financials['title'].value_counts()

In [ ]:
df_financials[df_financials['title'].duplicated() == True]

In [ ]:
# view selection of rows with duplicate titles
df_financials.sort_values('title').iloc[231:250, :]

In [ ]:
# Duplicate movie titles are due to movie being re-released years later. 
# Keep all rows and create new col 'title_yr'. 
df_financials['title_yr'] = df_financials['title'].str.cat(df_financials.year.astype(str), sep = '-')
df_financials.head()

In [ ]:
df_financials[df_financials['title_yr'].duplicated() == True]

In [ ]:
df_financials.sort_values('title').iloc[2017:2022, :]

In [ ]:
# Home movie - Double entry for 2009 release--unable to determine which one has accurate data, both appear to have notable outliers. Remove two 2009 entries.
df_financials = df_financials.drop([2019, 2020])

In [ ]:
df_financials[df_financials['title_yr'].duplicated() == True]

In [ ]:
df_financials.sort_values('title').iloc[2017:2022, :]

In [ ]:
# Create month col
df_financials['month_num'] = pd.DatetimeIndex(df_financials['release_date']).month
df_financials['month'] = pd.to_datetime(df_financials['month_num'], format='%m').dt.month_name().str.slice(stop=3)
df_financials.head()

In [ ]:
# Examine distribution of data

In [ ]:
display(df_financials.info())
display(df_financials[["production_budget_m", "domestic_gross_m", "worldwide_gross_m", "foreign_gross_m", "net_revenue_m", "ROI"]].apply(['mean', 'median', 'std']))
display(df_financials.quantile(q=0.75))
display(df_financials.quantile(q=0.85))
display(df_financials.quantile(q=0.90))
display(df_financials.quantile(q=0.95))
display(df_financials.quantile(q=0.99))

In [ ]:
# Domestic and worldwide gross revenue, net revenue, ROI are positively skewed--mean is larger than median, with large outliers distorting the mean.

In [ ]:
df_financials.boxplot(column=['worldwide_gross_m'])

In [ ]:
df_financials.boxplot(column=['ROI'])

In [ ]:
# Number of movies in 75th, 90th, 99th percentiles - By worldwide gross revenue
print(f"Number of movies in 75th percentile: {len(df_financials[df_financials['worldwide_gross_m']>1.044750e+02])}")    
print(f"Number of movies in 90th percentile: {len(df_financials[df_financials['worldwide_gross_m']>2.596100e+02])}")
print(f"Number of movies in 95th percentile: {len(df_financials[df_financials['worldwide_gross_m']>4.103150e+02])}")
print(f"Number of movies in 99th percentile: {len(df_financials[df_financials['worldwide_gross_m']>8.938990e+02])}")

In [ ]:
# Q1: What movies are the top gross revenue generators? (including outliers)

# Create new df - movies with gross revenue above 90th percentile
df_grossrev_99p = df_financials[df_financials.worldwide_gross_m >= 8.938658e+02].sort_values('worldwide_gross_m', ascending = False)
display(df_grossrev_99p.shape)

# top 25
df_grossrev_top25 = df_financials.sort_values('worldwide_gross_m', ascending = False).head(25)
display(df_grossrev_top25.shape)

In [ ]:
# Graph gross revenue movies

ax1 = df_grossrev_99p.plot.barh(x= 'title', 
                                y= 'worldwide_gross_m', 
                                color = 'blue', 
                                stacked=False, 
                                rot=0, 
                                figsize=(12,10), 
                                label = '_nolegend_')
ax1.set(
    title='Top Revenue Grossing Movies',
    xlabel='Millions $',
    ylabel='Movies')
ax1.invert_yaxis()

ax2 = df_grossrev_99p.plot.barh(x= 'title', 
                                y = 'ROIpct', 
                                color = 'green', 
                                stacked=False, 
                                rot=0, 
                                figsize=(12,10),
                                label = '_nolegend_')

ax2.set(
    title='ROI of Top Revenue Grossing Movies',
    xlabel='ROI (%)',
    ylabel='Movies')
ax2.invert_yaxis()

plt.style.use('ggplot')
plt.show()

In [ ]:
# Q2: Is movie industry profitable? Analysis w/o outliers and from year 2000 - present

# Create ROI df, remove NaN
df_roi = df_financials[df_financials['ROI'].notnull()]
display(df_financials.describe())
df_roi.describe()

In [ ]:
# Outlier detection
min_roi_threshold, max_roi_threshold = df_roi['ROI'].quantile([0.00, 0.90]).round(2)
min_roi_threshold, max_roi_threshold

In [ ]:
df_roi[df_roi['ROI'] > max_roi_threshold]

In [ ]:
# new DF without ROI outliers and with movies from year 2000 - present
df_roi = df_roi[(df_roi['ROI'] < max_roi_threshold) & 
                       (df_roi['ROI'] > min_roi_threshold) & 
                       (df_roi['year'] >= 2000)]
display(df_roi.sort_values('ROI', ascending = False).head(10))

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.hist(df_roi['ROI'], bins='auto', alpha = 0.5)
ax.set_title('Distribution of ROI')
ax.set_xlabel('ROI')
ax.set_ylabel('Number of Movies')
ax.axvline(df_roi['ROI'].mean(), color = 'black')
ax.axvline(df_roi['ROI'].median(), color = 'red')

In [ ]:
display(df_roi.ROI.apply(['mean', 'median', 'std']))

In [ ]:
df_roi.boxplot(column=['ROI'])

In [ ]:
#create new ROI buckets column 
def ROI_buckets(r):
    if r < 0: 
        return 'Not profitable'
    if r <= 1 : 
        return '0 - 100%'
    elif r <= 2: 
        return '100% - 200%'
    elif r <= 3: 
        return '200% - 300%'
    elif r <= 4: 
        return '300% - 400%'
    elif r <= 5: 
        return '400% - 500%'
    elif r <= 6: 
        return '500% - 600%'    
    elif r <= 7:
        return '600% - 700&'
    elif r <= 8:
        return '700% - 800%'
    else: 
        return 'Other'

df_roi['ROI_buckets'] = df_roi['ROI'].apply(ROI_buckets)
display(df_roi.head(10))
df_roi.info()

In [ ]:
#Distribution of ROI by year
df_roi_yr = df_roi.groupby('year')['ROI_buckets'].value_counts(normalize = True)
df_roi_yr.tail(10)

In [ ]:
# Analysis of ROI of movies in 90th percentile by ROI and after year 2000 (3,676 total movies)
df_roi_yr_median = df_roi.groupby(['year']).agg(['median'])
df_roi_yr_median

In [ ]:
# Graph median and mean ROI 

df_roi_yr_median['ROIpct'].plot.bar(figsize=(10,6), color = 'green')
plt.title('Median ROI - Movies in 90th percentile', fontsize = 12)
plt.xlabel('Year', fontsize = 12)
plt.ylabel('ROI', fontsize = 12)

plt.style.use('ggplot')

In [ ]:
# Graph ROI of movies in 90th percentile by ROI and after year 2000 (3,676 total movies)

#df_roi['ROI_buckets'].value_counts(normalize = True).plot.bar(figsize=(10,6), color = 'green')
#plt.title('ROI - Movies in 90th percentile', fontsize = 12)
#plt.xlabel('Worldwide gross revenue / production budget', fontsize = 12)
#plt.ylabel('Share of movies', fontsize = 12)

#plt.style.use('ggplot')

In [ ]:
df_roi_pct = ((df_roi.groupby(['year', 'ROI_buckets'])['ROI_buckets'].count()/df_roi.groupby(['year'])['ROI_buckets'].count()))*100
display(df_roi_pct.head(5))

In [ ]:
from matplotlib import cm
viridis = cm.get_cmap('viridis', 9)

In [ ]:
ax = df_roi_pct.unstack().plot.bar(stacked = True, figsize=(14,8), color=viridis.colors)

ax.set_title('ROI - Movies in 90th percentile', fontsize = 14)
ax.set_xlabel('Year', fontsize = 12)
ax.set_ylabel('Percent of movies (%)', fontsize = 12)
ax.yaxis.set_ticks(np.arange(0, 110, 10))
ax.tick_params(labeltop=False, labelright=True)

# Reverse legend order
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1.1,0.5))

plt.show()

In [ ]:
## Explore distributions and relationships
df_roi.plot.scatter(x = 'ROIpct',
                       y = 'production_budget', 
                       c = 'blue', 
                   figsize = (10,6))

from scipy.stats import linregress
x = df_roi.ROIpct
y = df_roi.production_budget
stats = linregress(x, y)

m = stats.slope
b = stats.intercept

# Set the linewidth on the regression line to 3px
plt.plot(x, m * x + b, color="red", linewidth=3)

plt.title('Relationship between ROI and Production Budget', fontsize = 12)
plt.xlabel('ROI (%)', fontsize = 12)
plt.ylabel('Production Budget ($)', fontsize = 12)

plt.show()

In [ ]:
df_roi.groupby('ROI_buckets').production_budget.median().plot.bar(figsize=(14,8))
plt.title('Production Budget', fontsize = 12)
plt.xlabel('ROI', fontsize = 12)
plt.ylabel('Production Budget ($)', fontsize = 12)

In [ ]:
df_roi_yr_median.reset_index(inplace = True)
df_roi_yr_median.head()

In [ ]:
yr = df_roi_yr_median[['year']]
worldwide_gross = df_roi_yr_median[['worldwide_gross_m']]
production_budget = df_roi_yr_median[['production_budget_m']]

df_roi_comp = df_roi_yr_median[['year','worldwide_gross_m','production_budget_m']]
df_roi_comp = df_roi_comp.rename(columns = {'year':'Year', 'worldwide_gross_m': 'Gross Revenue', 'production_budget_m': 'Production Budget'})
df_roi_comp.head()



In [ ]:
# Graph gross revenue and production budget by year

df_roi_comp.plot(x="Year", y=["Production Budget", "Gross Revenue"], kind="bar", figsize=(14,8))
plt.title('Gross Revenue and Production Budget', fontsize = 12)
plt.xlabel('Year', fontsize = 12)
plt.yticks(np.arange (0, 75, 5))
plt.ylabel('Millions ($)', fontsize = 12)

plt.legend()

In [ ]:
df_roi_comp_from2014 = df_roi_yr_median[['year','worldwide_gross_m','production_budget_m']][df_roi_yr_median['year']>=2014]
df_roi_comp_from2014.head()

In [ ]:
df_roi_comp_from2014.plot.scatter(x = 'worldwide_gross_m',
                       y = 'production_budget_m', 
                       c = 'blue', 
                   figsize = (10,6))

from scipy.stats import linregress
x = df_roi_comp_from2014.worldwide_gross_m
y = df_roi_comp_from2014.production_budget_m
#stats = linregress(x, y)

m = stats.slope
b = stats.intercept

# Set the linewidth on the regression line to 3px
#plt.plot(x, m * x + b, color="red", linewidth=3)

plt.title('Relationship between Gross Revenue and Production Budget', fontsize = 12)
plt.xlabel('Gross Revenue', fontsize = 12)
plt.ylabel('Production Budget', fontsize = 12)

plt.show()

In [ ]:
## ROI and year movie released
df_roi.plot.scatter(x = 'year',
                       y = 'ROI', 
                       c = 'blue', 
                   figsize = (10,6))

plt.title('Relationship between ROI and Year', fontsize = 12)
plt.xlabel('Year', fontsize = 12)
plt.ylabel('ROI (X)', fontsize = 12)

plt.show()

In [ ]:
# ROI and month movie released
df_roi.plot.scatter(x = 'ROIpct',
                       y = 'month', 
                       c = 'blue', 
                   figsize = (10,6))

x = df_roi.ROIpct
y = df_roi.month
stats = linregress(x, y)

m = stats.slope
b = stats.intercept

# Set the linewidth on the regression line to 3px
plt.plot(x, m * x + b, color="red", linewidth=3)

plt.title('Relationship between ROI and Release Month', fontsize = 12)
plt.xlabel('ROI (%)', fontsize = 12)
plt.ylabel('Month', fontsize = 12)

plt.show()

In [ ]:
df_roi.head()

In [ ]:
# NET REVENUE

In [ ]:
# Number of movies in 75th, 99th percentiles - By worldwide net revenue
print(f"Number of movies in 75th percentile: {len(df_revenue[df_revenue['net_revenue']>6.735617e+07])}")    
print(f"Number of movies in 99.5 percentile: {len(df_revenue[df_revenue['net_revenue']>3.187214e+08])}")
print(f"Number of movies in 99.9 percentile: {len(df_revenue[df_revenue['net_revenue']>7.646151e+08])}")

In [ ]:
# Identify top movies by worldwide net revenue
df_revenue.sort_values(by = ['net_revenue'], axis=0, ascending=[False], inplace=False,
               na_position='last', ignore_index=True)[['title', 'net_revenue', 'ROI', 'production_budget']].head()

In [ ]:
# Identify movies with worldwide net revenue in 99.9 percentile

# Create new df
df_nr_999p = df_revenue[df_revenue.net_revenue >= 7.646151e+08].sort_values('net_revenue', ascending = False)
df_nr_999p.head()

# how to use .loc?
#nr_99_3p = df_revenue['net_revenue'].loc[lambda x: x>8.407595e+08].sort_values(ascending = False, ignore_index = True)
#nr_99_3p = df_revenue[['title','net_revenue']]['net_revenue'].loc[lambda x: x>8.407595e+08].sort_values(ascending = False, ignore_index = True)

In [ ]:
# Graph net revenue and ROI of movies in 99.9 percentile by net revenue

ax1 = df_nr_999p.plot.barh(x= 'title', y= 'net_revenue', color = 'blue', stacked=False, rot=0, figsize=(12,10))
ax1.set(
    title='Net Revenue - Movies in 99th Percentile',
    xlabel='Billions $',
    ylabel='Movies')
ax1.invert_yaxis()

ax2 = df_nr_999p.plot.barh(x= 'title', y = 'ROI', color = 'green', stacked=False, rot=0, figsize=(12,10))
ax2.set(
    title='ROI - Movies in 99th Percentile by Net Revenue',
    xlabel='ROI (x)',
    ylabel='Movies')
ax2.invert_yaxis()

plt.style.use('ggplot')
plt.show()

In [ ]:
# merge df_ROI_v2 with df4 (has info on genres, run time)

In [ ]:
## Explore distributions and relationships
pd.plotting.scatter_matrix(df_revenue[['production_budget', 'domestic_gross', 'foreign_gross', 'worldwide_net_revenue']], figsize=(6,6));
plt.show()

In [ ]:
# Join with df4, which has info on genres, runtime, etc.
df4.head()

In [ ]:
df4 = df4.rename(columns={"primary_title": "title"})
df4.head()

In [ ]:
df4['title_yr'] = df4['title'].str.cat(df4.start_year.astype(str), sep = '-')
df4.head()

In [ ]:
df4.title.value_counts()

In [ ]:
df4.title_yr.value_counts()

In [ ]:
df_main = df_revenue.join(df4, how='left', rsuffix='_right')

In [ ]:
# QUESTIONS

In [ ]:
# glyphs

In [ ]:
## Clean up: Rename columns, convert dtypes to prepare dfs for joins and analysis

In [ ]:
df2 = df2.rename(columns={"nconst": "name_id"})
df2.head()

In [ ]:
df4 = df4.rename(columns={"tconst": "title_id"})
df4.head()

In [ ]:
df5 = df5.rename(columns={"tconst": "title_id"})
df5.head()

In [ ]:
df6 = df6.rename(columns={"tconst": "title_id", "nconst": "name_id"})
df6.head()

In [ ]:
df7 = df7.rename(columns={"tconst": "title_id"})
df7.head()

In [ ]:
df8['theater_date'] = pd.to_datetime(df8['theater_date'])
df8['dvd_date'] = pd.to_datetime(df8['dvd_date'])
df8.info()

In [ ]:
df8 = df8.rename(columns={"runtime": "runtime_mins"})
df8['runtime_mins'] = df8['runtime_mins'].str.rstrip('minutes')
df8.head()

In [ ]:
df10['release_date'] = pd.to_datetime(df10['release_date'])
df10.info()

In [ ]:
## Calculate worldwide_net_revenue and foreign_gross (revenue generated outside of US)
df11['worldwide_net_revenue'] = df11.worldwide_gross - df11.production_budget
df11['foreign_gross'] = df11.worldwide_gross - df11.domestic_gross

In [ ]:
## Explore distributions and relationships
pd.plotting.scatter_matrix(df11[['production_budget', 'domestic_gross', 'foreign_gross','worldwide_gross', 'worldwide_net_revenue']], figsize=(6,6));
plt.show()

In [ ]:
df11.apply(['mean', 'median', 'std'])

In [ ]:
# Explore distribution of worldwide_net_revenue
fig, ax = plt.subplots(figsize=(6,6))
ax.hist(df11['worldwide_net_revenue'], bins='auto', alpha = 0.5)
ax.set_title('Distribution of Worldwide Net Revenue')
ax.set_xlabel('Net Revenue ($ Billions)')
ax.set_ylabel('Number of Movies')
ax.axvline(df11['worldwide_net_revenue'].mean(), color = 'black')
ax.axvline(df11['worldwide_net_revenue'].median(), color = 'red')

In [ ]:
df11['worldwide_net_revenue'].describe()

In [ ]:
# Worldwide net revenue is positively skewed--mean is larger than median, with large outliers distorting the mean.
# Identify movies with above-median worldwide net revenue
above_median_worldwide_net_revenue = []

#df11.query('worldwide_net_revenue'>=8.550286e+06)

movies = df11.movie.values
#ww_net_rev = df11.worldwide_net_revenue.values

if df11['worldwide_net_revenue'] >= 8.550286e+06:
    above_median_worldwide_net_revenue.append()

above_median_worldwide_net_revenue

In [ ]:
# Explore distribution of worldwide_net_revenue

fig, ax = plt.subplots(4, figsize=(10,8))
ax1 = plt.subplot(1,4,1)
ax1.hist(df_revenue['net_revenue'], bins='auto', color = 'green', alpha = 0.5)
ax1.set_title('Net Revenue', fontsize = 10)
ax1.set_xlabel('Dollars $', fontsize = 10)
ax1.set_ylabel('Number of Movies', fontsize = 10)
ax1.axvline(df_revenue['net_revenue'].mean(), color = 'black')

ax2 = plt.subplot(1,4,2)
ax2.hist(df_revenue['domestic_gross'], bins='auto', color = 'green', alpha = 0.5)
ax2.set_title('Domestic Gross Revenue' , fontsize = 10)
ax2.set_xlabel('Dollars $', fontsize = 10)
ax2.axvline(df_revenue['domestic_gross'].mean(), color = 'black')

ax3 = plt.subplot(1,4,3)
ax3.hist(df_revenue['foreign_gross'], bins='auto', color = 'green', alpha = 0.5)
ax3.set_title('Foreign Gross Revenue', fontsize = 10)
ax3.set_xlabel('Dollars $', fontsize = 10)
ax3.axvline(df_revenue['foreign_gross'].mean(), color = 'black')

ax4 = plt.subplot(1,4,4)
ax4.hist(df_revenue['production_budget'], bins='auto', color = 'green', alpha = 0.5)
ax4.set_title('Production Budget', fontsize = 10)
ax4.set_xlabel('Dollars $', fontsize = 10)
ax4.axvline(df_revenue['production_budget'].mean(), color = 'black')

#plt.style.use('ggplot')
plt.show()